In [1]:
%reload_ext autoreload
%autoreload 2

# Introduction to LMI
- **Goal:** Get familiar with the basic operations, get a feel for how the training / searching process operates.
- **Table of Contents:**
    - [1. Load the data](#1.-Load-the-data)
    - [2. Build the index](#2.-Build-the-index)
    - [3. Load additional data for search](#3.-Load-search-data)
    - [4. Search in the index](#4.-Search-in-the-index)
- **Author:** T. Slanináková, `xslanin@fi.muni.cz`
- **Date:** 2023-09-12

In [2]:
import logging
import numpy as np
np.random.seed(2023)

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s][%(levelname)-5.5s][%(name)-.20s] %(message)s'
)
LOG = logging.getLogger(__name__)

# 1. Load the data
The data are from SISAP 2023 indexing challenge (LAION dataset). There are `100K`, `300K`, and `10M` versions (also `100M`, but that one wasn't tested with LMI). The queries are not included in the data (they are outside of the dataset).

In [4]:
import os
from urllib.request import urlretrieve
from pathlib import Path
import h5py

def download(src, dst):
    if not os.path.exists(dst):
        os.makedirs(Path(dst).parent, exist_ok=True)
        LOG.info('downloading %s -> %s...' % (src, dst))
        urlretrieve(src, dst)

def prepare(kind, size):
    url = "https://sisap-23-challenge.s3.amazonaws.com/SISAP23-Challenge"
    task = {
        "query": f"{url}/public-queries-10k-{kind}.h5",
        "dataset": f"{url}/laion2B-en-{kind}-n={size}.h5",
    }

    for version, url in task.items():
        target_path = os.path.join("data", kind, size, f"{version}.h5")
        download(url, target_path)
        assert os.path.exists(target_path), f"Failed to download {url}"

In [5]:
config = {
    # get the smallest version of the LAION dataset
    'dataset': 'pca32v2',
    'emb': 'pca32',
    'size': '100K',
    # n. of nearest neighbors
    'k': 10,
    # normalize the data to be able to use K-Means
    'preprocess': True
}

In [6]:
# download the data
prepare(config['dataset'], config['size'])

def get_data(data_part, **config):
    return np.array(
        h5py.File(
            os.path.join(
                'data',
                config['dataset'],
                config['size'],
                data_part
            ),
            'r'
        )[config['emb']]
    )

# load the data    
data = get_data("dataset.h5", **config)
queries = get_data("query.h5", **config)
data.shape, queries.shape

((100000, 32), (10000, 32))

## 1.2. Pre-process the data
The default distance metric for LAION dataset is the cosine distance. In order for us to use K-Means for partitioning (which operates only with Euclidean distances), we need to **normalize the data to unit length** (i.e., a single vector will sum up to 1). Data normalized like this can continue to be used with euclidean distance.

In [7]:
# data characteristic before:
sum(data[0])

0.4463985259644687

In [8]:
from sklearn import preprocessing
if config['preprocess']:
    data = preprocessing.normalize(data)
    queries = preprocessing.normalize(queries)

In [9]:
# data characteristics after
sum(data[0])

1.004468702711165

In [10]:
import pandas as pd
# data to pandas
data = pd.DataFrame(data)
# index from one (needed to fit the evaluation procedure later)
data.index += 1

# 2. Build the index

In [3]:
from li.BuildConfiguration import BuildConfiguration
from li.clustering import algorithms
from li.LearnedIndexBuilder import LearnedIndexBuilder

[2023-10-04 09:30:44,803][INFO ][faiss.loader] Loading faiss with AVX2 support.
[2023-10-04 09:30:47,167][INFO ][faiss.loader] Successfully loaded faiss with AVX2 support.
[2023-10-04 09:32:33,885][INFO ][numexpr.utils] Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2023-10-04 09:32:33,886][INFO ][numexpr.utils] Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [16]:
n_categories = [10, 10]

build_config = BuildConfiguration(
    # which clustering algorithm to use
    algorithms['faiss_kmeans'],
    # how many epochs to train for
    200,
    # what model to use (see li/model.py
    'MLP',
    # what learning rate to use
    0.01,
    # how many categories at what level to build LMI for
    # 10, 10 results in 100 buckets in total
    n_categories
)

In [18]:
%%time
builder = LearnedIndexBuilder(data, build_config)
li, data_prediction, n_buckets_in_index, build_t, cluster_t = builder.build()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:54<00:00, 17.42s/it]

CPU times: user 5min 38s, sys: 1.46 s, total: 5min 40s
Wall time: 5min 52s


In [20]:
LOG.info(f"Total number of buckets in the index: {n_buckets_in_index}")
LOG.info(f"Cluster time: {cluster_t}")
LOG.info(f"Pure build time: {build_t}")

[2023-10-04 09:58:00,083][INFO ][__main__] Total number of buckets in the index: 100
[2023-10-04 09:58:00,085][INFO ][__main__] Cluster time: 0.9519636631011963
[2023-10-04 09:58:00,086][INFO ][__main__] Pure build time: 352.65329146385193


In [21]:
# n_levels == len(n_categories)
# `data_prediction` (dimensions: n of data points x n_levels)
#   stores assignment of every object to a bucket
n_levels = len(n_categories)
assert data_prediction.shape == ((data.shape[0], len(n_categories)))

In [22]:
data_prediction[:2]

array([[8, 7],
       [3, 2]])

# 3. Load search data
In this specific dataset, we use data that has been subject to dimensionality reduction (hence `pca32v2` in `config['dataset']`). This is nice, because it allows us to train faster (on 32-dim. vectors vs. 768) and have faster navigation during search. However during the sequential search phase on the candidate set, we reach better accuracy when searching with the non-reduced set.

Let's load the full dataset now:

In [23]:
config['dataset'] = 'clip768v2'
config['emb'] = 'emb'

prepare(config['dataset'], config['size'])
data_search = get_data("dataset.h5", **config)
queries_search = get_data("query.h5", **config)

data_search = pd.DataFrame(data_search)
data_search.index += 1

data_search.shape, queries_search.shape

[2023-10-04 09:59:08,370][INFO ][__main__] downloading https://sisap-23-challenge.s3.amazonaws.com/SISAP23-Challenge/public-queries-10k-clip768v2.h5 -> data/clip768v2/100K/query.h5...
[2023-10-04 09:59:11,056][INFO ][__main__] downloading https://sisap-23-challenge.s3.amazonaws.com/SISAP23-Challenge/laion2B-en-clip768v2-n=100K.h5 -> data/clip768v2/100K/dataset.h5...


((100000, 768), (10000, 768))

# 4. Search in the index

In [42]:
# specify the stop condition
bucket=10
# specify the n. of neighbors
k=10

In [43]:
%%time
dists, nns, measured_time = li.search(
    # the 'navigation' data
    data_navigation=data,
    queries_navigation=queries,
    # the 'sequential filtering' data
    data_search=data_search,
    queries_search=queries_search,
    # mapping of object -> bucket
    data_prediction=data_prediction,
    # n. of categories present in index
    n_categories=n_categories,
    # stop condition for the search
    n_buckets=bucket,
    # number of nearest neighbors we're interested in
    k=k
)

[2023-10-04 10:07:00,406][INFO ][li.LearnedIndex.Lear] Precomputed bucket order time: 0.4756293296813965
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 180.81it/s]

CPU times: user 7.44 s, sys: 55.5 ms, total: 7.49 s
Wall time: 7.75 s


In [44]:
# Time to search (broken down into various search parts)
measured_time

defaultdict(float,
            {'inference': 0.05044746398925781,
             'search_within_buckets': 7.249154806137085,
             'seq_search': 4.535206317901611,
             'sort': 0.0,
             'search': 7.75262188911438})

In [27]:
# matrix of the nearest neighbors (`k` for each query)
nns.shape

(10000, 10)

In [28]:
nns[:2]

array([[79172, 15735, 22337, 74173, 41079, 38159, 69015, 92811, 79896,
        13236],
       [14347, 82848, 79302, 85923,  6016, 67067, 54566, 34591, 11620,
        53783]], dtype=uint32)

In [29]:
# matrix of distances to the closest neighbors (`k` for each query)
dists.shape

(10000, 10)

In [30]:
dists[:2]

array([[0.27291209, 0.30623567, 0.3131932 , 0.32404494, 0.33161247,
        0.33278447, 0.34032881, 0.34535122, 0.35354602, 0.36600691],
       [0.19766825, 0.21139383, 0.22871637, 0.23902297, 0.25272477,
        0.25969118, 0.2700808 , 0.2767331 , 0.27809215, 0.28464031]])

# 5. Evaluate the search performance

In [33]:
def get_groundtruth(size="100K"):
    url = f"https://sisap-23-challenge.s3.amazonaws.com/SISAP23-Challenge/laion2B-en-public-gold-standard-v2-{size}.h5"

    out_fn = os.path.join("data", f"groundtruth-{size}.h5")
    download(url, out_fn)
    gt_f = h5py.File(out_fn, "r")
    true_I = np.array(gt_f['knns'])
    gt_f.close()
    return true_I

gt = get_groundtruth(config['size'])

[2023-10-04 10:03:59,376][INFO ][__main__] downloading https://sisap-23-challenge.s3.amazonaws.com/SISAP23-Challenge/laion2B-en-public-gold-standard-v2-100K.h5 -> data/groundtruth-100K.h5...


In [36]:
def get_recall(I, gt, k):
    assert k <= I.shape[1]
    assert len(I) == len(gt)

    n = len(I)
    recall = 0
    for i in range(n):
        recall += len(set(I[i, :k]) & set(gt[i, :k]))
    return recall / (n * k)

In [45]:
recall = get_recall(nns, gt, k)
recall

0.87099